In [1]:
import warnings


warnings.simplefilter("ignore")

In [2]:
import numpy as np
import pandas as pd

import lightgbm as lgb

In [3]:
from unbiasedlambdamart.calculator import Calculator

## Testing get_gradients

In [4]:
from unbiasedlambdamart.lambdaobj import get_unbiased_gradients_fixed_t
from unbiasedlambdamart.objective import MIN_ARG, MAX_ARG
from unbiasedlambdamart.objective import DatasetWithCalculatorRanks0AndT

In [5]:
c = [1, 0, 0, 1, 0]
preds = [0.4, 0.5, 0.8, 0.1, -1.0]
ranks0 = [1, 2, 0, 0, 1]
groups = [3, 2]
discounts = [1, 1 / np.log2(3), 1 / np.log2(4)]
t_plus = [1, 0.5, 1 / 3]

def get_grad(pred_i, pred_j, rank0_i, pos_i, pos_j):
    abs_delta_ndcg = np.abs(discounts[pos_i] - discounts[pos_j])
    abs_delta_ndcg /= (0.01 + np.abs(pred_i - pred_j))
    return -2.0 / (1 + np.exp(2.0 * (pred_i - pred_j))) * abs_delta_ndcg / t_plus[rank0_i]

dataset = DatasetWithCalculatorRanks0AndT(
    3,
    np.array(ranks0) + 1,
    1.0,
    data=np.zeros((5, 1)),
    label=c,
    group=groups,
    free_raw_data=False
)
grad = np.zeros(len(preds))
hess = np.zeros(len(preds))
get_unbiased_gradients_fixed_t(
    np.ascontiguousarray(dataset.label, dtype=np.double), 
    np.ascontiguousarray(preds),
    np.ascontiguousarray(dataset.t_plus),
    np.ascontiguousarray(dataset.t_minus),
    np.ascontiguousarray(dataset.ranks0),
    len(preds),
    np.ascontiguousarray(groups),
    np.ascontiguousarray(dataset.calculator.query_boundaries),
    len(dataset.calculator.query_boundaries) - 1,
    np.ascontiguousarray(dataset.calculator.discounts),
    np.ascontiguousarray(dataset.calculator.inverse_max_dcgs),
    np.ascontiguousarray(dataset.calculator.sigmoids),
    len(dataset.calculator.sigmoids),
    MIN_ARG,
    MAX_ARG,
    dataset.calculator.idx_factor,
    np.ascontiguousarray(grad), 
    np.ascontiguousarray(hess)
)

assert np.allclose(
    grad,
    [
        get_grad(0.4, 0.5, 1, 2, 1) + get_grad(0.4, 0.8, 1, 2, 0),
        -get_grad(0.4, 0.5, 1, 2, 1),
        -get_grad(0.4, 0.8, 1, 2, 0),
        get_grad(0.1, -1.0, 0, 1, 0),
        -get_grad(0.1, -1.0, 0, 1, 0)
    ]
)

In [6]:
from unbiasedlambdamart.lambdaobj import get_unbiased_gradients
from unbiasedlambdamart.objective import DatasetWithCalculatorRanks0AndP

In [7]:
c = [1, 0, 0, 1, 0]
preds = [0.4, 0.5, 0.8, 0.1, -1.0]
ranks0 = [1, 2, 0, 0, 1]
groups = [3, 2]
discounts = [1, 1 / np.log2(3), 1 / np.log2(4)]
t_plus = np.array([1, 0.5, 1 / 3])
t_minus = np.ones(3)
t_plus_copy = t_plus.copy()
t_minus_copy = t_minus.copy()

def get_grad(pred_i, pred_j, rank0_i, rank0_j, pos_i, pos_j):
    abs_delta_ndcg = np.abs(discounts[pos_i] - discounts[pos_j])
    abs_delta_ndcg /= (0.01 + np.abs(pred_i - pred_j))
    return -2.0 / (1 + np.exp(2.0 * (pred_i - pred_j))) * abs_delta_ndcg / t_plus[rank0_i] / t_minus[rank0_j]

def get_t(pred_i, pred_j, rank0_i, rank0_j, pos_i, pos_j, t_plus, t_minus):
    abs_delta_ndcg = np.abs(discounts[pos_i] - discounts[pos_j])
    abs_delta_ndcg /= (0.01 + np.abs(pred_i - pred_j))
    loss = np.log(1 + np.exp(-2.0 * (pred_i - pred_j))) * abs_delta_ndcg 
    return loss / t_minus[rank0_j], loss / t_plus[rank0_i]

dataset = DatasetWithCalculatorRanks0AndP(
    3,
    np.array(ranks0) + 1,
    1.0,
    data=np.zeros((5, 1)),
    label=c,
    group=groups,
    free_raw_data=False
)
grad = np.zeros(len(preds))
hess = np.zeros(len(preds))

get_unbiased_gradients(
    np.ascontiguousarray(dataset.label, dtype=np.double), 
    np.ascontiguousarray(preds),
    np.ascontiguousarray(dataset.ranks0),
    len(preds),
    np.ascontiguousarray(groups),
    np.ascontiguousarray(dataset.calculator.query_boundaries),
    len(dataset.calculator.query_boundaries) - 1,
    np.ascontiguousarray(dataset.calculator.discounts),
    np.ascontiguousarray(dataset.calculator.inverse_max_dcgs),
    np.ascontiguousarray(dataset.calculator.sigmoids),
    len(dataset.calculator.sigmoids),
    MIN_ARG,
    MAX_ARG,
    dataset.calculator.idx_factor,
    np.ascontiguousarray(dataset.calculator.logs),
    len(dataset.calculator.logs),
    MIN_ARG,
    MAX_ARG,
    dataset.calculator.idx_factor,
    dataset.p,
    dataset.calculator.max_rank,
    np.ascontiguousarray(grad), 
    np.ascontiguousarray(hess),
    np.ascontiguousarray(t_plus_copy),
    np.ascontiguousarray(t_minus_copy)
)

assert np.allclose(
    grad,
    [
        get_grad(0.4, 0.5, 1, 2, 2, 1) + get_grad(0.4, 0.8, 1, 0, 2, 0),
        -get_grad(0.4, 0.5, 1, 2, 2, 1),
        -get_grad(0.4, 0.8, 1, 0, 2, 0),
        get_grad(0.1, -1.0, 0, 1, 1, 0),
        -get_grad(0.1, -1.0, 0, 1, 1, 0)
    ]
)

new_t_plus = np.zeros(len(t_plus))
new_t_minus = np.zeros(len(t_minus))
tp012, tm012 = get_t(0.4, 0.5, 1, 2, 2, 1, t_plus, t_minus)
tp010, tm010 = get_t(0.4, 0.8, 1, 0, 2, 0, t_plus, t_minus)
tp101, tm101 = get_t(0.1, -1.0, 0, 1, 1, 0, t_plus, t_minus)
new_t_plus[1] += tp012
new_t_minus[2] += tm012
new_t_plus[1] += tp010
new_t_minus[0] += tm010
new_t_plus[0] += tp101
new_t_minus[1] += tm101
new_t_plus = np.power(new_t_plus / new_t_plus[0], 1 / 2)
new_t_minus = np.power(new_t_minus / new_t_minus[0], 1 / 2)

assert np.allclose(new_t_plus, t_plus_copy)
assert np.allclose(new_t_minus, t_minus_copy)

## Generating Data

In [8]:
np.random.seed(0)

In [9]:
def generate_data(n_positions, coef, n_requests):
    """
    This function is used for simulating the data. We generate n_requests result pages
    with n_positions positions each. A logistic regression model is used for generating 
    interactions. The coefficients are provided via the coef parameter. The number of
    features is inferred from its length. Features are simulated as standard normal random
    variables. Position biases are set to 1/k, where k is the position number.
    :param page_len: the number of positions on each result page
    :param coef: a matrix defining the two logistic regression models for generating
                 interactions
    :param n_requests: the number of requests/queries/result pages
    :param position_biases
    :returns: a pandas.DataFrame having n_requests * n_positions rows 
              and the following columns:
                request_id,
                feature_1, ..., feature_m (where m is coef.shape[1]),
                relevance (click probability given that the position was observed),
                c (click indicator)
    """
    n_features = len(coef)
    feature_names = ["feature_%i" % i for i in range(1, n_features + 1)]
    data = pd.DataFrame(
        np.concatenate(
            [
                np.repeat(range(n_requests), n_positions)[:, None],
                np.array(list(range(1, n_positions + 1)) * n_requests)[:, None],
                np.random.normal(0, 1, (n_requests * n_positions, n_features))
            ],
            axis=1
        ),
        columns=["request_id", "position"] + feature_names
    )
    view_probs = 1 / np.arange(1, n_positions + 1)
    view_probs = view_probs / np.concatenate([[1.0], view_probs[:-1]])
    view_indicators = np.random.binomial(1, list(view_probs) * n_requests)
    z = np.dot(data[feature_names].values, coef) - 4.0
    data["relevance"] = 1 / (1 + np.exp(-z))
    c = np.random.binomial(1, data.relevance)
    data["c"] = np.zeros(len(data))
    for i in range(n_requests):
        for j in range(n_positions):
            idx = i * n_positions + j
            if not view_indicators[idx]:
                break
            data["c"].iloc[idx] = c[idx]
    data["viewed"] = view_indicators
    data["_c"] = c
    return data


def drop_requests_with_no_interactions(data, interaction_col):
    interaction_requests = set(data.loc[data[interaction_col] > 0].request_id)
    return data.loc[data.request_id.isin(interaction_requests)]

In [10]:
COEF = [1.0, -1.0]
N_POSITIONS = 10
MAX_NDCG_POS = 10
N_TRAIN = 50000

In [11]:
train_data = generate_data(N_POSITIONS, [1.0, -1.0], N_TRAIN)

## Training

In [12]:
lgb_params = {
    "num_trees": 10,
    "objective": "lambdarank",
    "max_position": MAX_NDCG_POS, 
    "metric": "ndcg",
    "eval_at": MAX_NDCG_POS
}

In [13]:
class DatasetWithCalculatorAndRanks0(lgb.Dataset):
    def __init__(self, max_ndcg_pos, ranks, *args, **kwargs):
            lgb.Dataset.__init__(self, *args, **kwargs)
            self.calculator = Calculator(self.label, self.get_group(), max_ndcg_pos)
            self.ranks0 = (ranks - 1).astype(int)

In [14]:
train_data = drop_requests_with_no_interactions(train_data, "c")
train_dataset = DatasetWithCalculatorAndRanks0(
    MAX_NDCG_POS,
    train_data.position.values,
    train_data.drop(["request_id", "position", "c", "relevance", "viewed", "_c"], axis=1),
    label=train_data.c.values,
    group=[N_POSITIONS] * train_data.request_id.nunique(),
    free_raw_data=False
)

### With a biased objective

In [15]:
def get_gradient_for_one_query(
    gains,
    preds,
    start,
    end,
    inverse_max_dcg,
    grad, 
    hess, 
    calculator
):
    sorted_idx = np.argsort(-preds[start:end])
    should_adjust = preds[start + sorted_idx[0]] != preds[start + sorted_idx[-1]]
    for i in range(len(sorted_idx)):
        high = sorted_idx[i]
        gain_high = gains[start + high]
        for j in range(len(sorted_idx)):
            low = sorted_idx[j]
            gain_low = gains[start + low]
            if gain_high > gain_low:
                score_high = preds[start + high]
                score_low = preds[start + low]
                score_diff = score_high - score_low
                p_lambda = calculator.get_sigmoid(score_diff)
                p_hess = p_lambda * (2.0 - p_lambda)
                gain_diff = gain_high - gain_low
                paired_discount = calculator.discounts[1 + i] - calculator.discounts[1 + j]
                abs_delta_ndcg = np.abs(gain_diff * paired_discount * inverse_max_dcg)
                if should_adjust == 1:
                    abs_delta_ndcg /= (0.01 + np.abs(score_diff))
                p_lambda *= -abs_delta_ndcg
                p_hess *= 2 * abs_delta_ndcg
    
                grad[start + high] += p_lambda
                grad[start + low] -= p_lambda
                hess[start + high] += p_hess
                hess[start + low] += p_hess
                
                
def get_gradients(gains, preds, groups, grad_for_one_query_func, calculator):
    grad = [0] * len(gains)
    hess = [0] * len(gains)
    
    query_boundaries = calculator.query_boundaries   

    for i in range(len(query_boundaries) - 1):
        grad_for_one_query_func(gains, 
                                preds, 
                                query_boundaries[i], 
                                query_boundaries[i + 1], 
                                calculator.inverse_max_dcgs[i],
                                grad, 
                                hess,
                                calculator)

    return grad, hess


def lambdarank_objective(preds, train_data):
    groups = train_data.get_group()
    calculator = train_data.calculator
    
    if len(groups) == 0:
        raise Error("Group/query data should not be empty.")
    else:
        grad, hess = get_gradients(
            train_data.label,
            preds,
            groups,
            get_gradient_for_one_query,
            calculator
        )

        return grad, hess

In [16]:
def fit_original(dataset, verbose_eval=True):
    lgb.train(
        params=lgb_params, 
        train_set=dataset,
        valid_sets=[dataset],
        verbose_eval=verbose_eval
    )    


fit_original(train_dataset)

[1]	training's ndcg@10: 0.727797
[2]	training's ndcg@10: 0.72946
[3]	training's ndcg@10: 0.730345
[4]	training's ndcg@10: 0.730983
[5]	training's ndcg@10: 0.731797
[6]	training's ndcg@10: 0.732035
[7]	training's ndcg@10: 0.732415
[8]	training's ndcg@10: 0.732267
[9]	training's ndcg@10: 0.732783
[10]	training's ndcg@10: 0.733984


In [17]:
def fit_custom_objective(dataset, verbose_eval=True):
    lgb.train(
        params=lgb_params, 
        train_set=dataset,
        valid_sets=[dataset],
        verbose_eval=verbose_eval,
        fobj=lambdarank_objective
    )    


fit_custom_objective(train_dataset)

[1]	training's ndcg@10: 0.727797
[2]	training's ndcg@10: 0.72946
[3]	training's ndcg@10: 0.730345
[4]	training's ndcg@10: 0.730983
[5]	training's ndcg@10: 0.731797
[6]	training's ndcg@10: 0.732035
[7]	training's ndcg@10: 0.732415
[8]	training's ndcg@10: 0.732267
[9]	training's ndcg@10: 0.732783
[10]	training's ndcg@10: 0.733984


### Using a python implementation of the unbiased objective

#### With updating t_plus and t_minus

In [18]:
def get_unbiased_gradient_for_one_query(
    gains,
    preds,
    cur_t_plus,
    cur_t_minus,
    next_t_plus,
    next_t_minus,
    ranks0,
    start, 
    end,
    inverse_max_dcg,
    grad, 
    hess, 
    calculator
):
    sorted_idx = np.argsort(-preds[start:end])
    should_adjust = preds[start + sorted_idx[0]] != preds[start + sorted_idx[-1]]
    for i in range(len(sorted_idx)):
        high = sorted_idx[i]
        gain_high = gains[start + high]
        for j in range(len(sorted_idx)):
            low = sorted_idx[j]
            gain_low = gains[start + low]
            if gain_high > gain_low:
                score_high = preds[start + high]
                score_low = preds[start + low]
                rank_high = ranks0[start + high]
                rank_low = ranks0[start + low]

                score_diff = score_high - score_low
                p_lambda = calculator.get_sigmoid(score_diff)
                p_hess = p_lambda * (2.0 - p_lambda)
                gain_diff = gain_high - gain_low
                paired_discount = calculator.discounts[1 + i] - calculator.discounts[1 + j]
                abs_delta_ndcg = np.abs(gain_diff * paired_discount * inverse_max_dcg)
                if should_adjust == 1:
                    abs_delta_ndcg /= (0.01 + np.abs(score_diff))
                p_lambda *= -abs_delta_ndcg
                p_hess *= 2 * abs_delta_ndcg
                
                p_lambda /= cur_t_plus[rank_high] * cur_t_minus[rank_low]
                p_hess /= cur_t_plus[rank_high] * cur_t_minus[rank_low]
    
                grad[start + high] += p_lambda
                grad[start + low] -= p_lambda
                hess[start + high] += p_hess
                hess[start + low] += p_hess
                
                loss = calculator.get_log(score_diff) * abs_delta_ndcg
                next_t_plus[rank_high] += loss / cur_t_minus[rank_low]
                next_t_minus[rank_low] += loss / cur_t_plus[rank_high]  
                
                
def get_unbiased_gradients(
    gains, 
    preds, 
    cur_t_plus, 
    cur_t_minus,
    ranks0,
    groups, 
    grad_for_one_query_func, 
    calculator,
    p
):
    grad = [0] * len(gains)
    hess = [0] * len(gains)
    next_t_plus = [0] * len(cur_t_plus)
    next_t_minus = [0] * len(cur_t_minus)
    
    query_boundaries = calculator.query_boundaries   

    for i in range(len(query_boundaries) - 1):
        grad_for_one_query_func(gains, 
                                preds,
                                cur_t_plus,
                                cur_t_minus,
                                next_t_plus,
                                next_t_minus,
                                ranks0,
                                query_boundaries[i], 
                                query_boundaries[i + 1], 
                                calculator.inverse_max_dcgs[i],
                                grad, 
                                hess,
                                calculator)
    
    next_t_plus = np.power(next_t_plus / next_t_plus[0], 1 / (1 + p))
    next_t_minus = np.power(next_t_minus / next_t_minus[0], 1 / (1 + p))
    
    return grad, hess, next_t_plus, next_t_minus


def unbiased_lambdarank_objective(preds, train_data):
    """
    Uses global variables t_plus and t_minus.
    """
    global t_plus, t_minus
    
    groups = train_data.get_group()

    if len(groups) == 0:
        raise Error("Group/query data should not be empty.")
    else:
        grad, hess, next_t_plus, next_t_minus = get_unbiased_gradients(
            train_data.label,
            preds,
            t_plus,
            t_minus,
            train_data.ranks0,
            groups,
            get_unbiased_gradient_for_one_query,
            train_data.calculator,
            0.0
        )
        t_plus, t_minus = next_t_plus, next_t_minus
        
        return grad, hess

In [19]:
def fit_custom_objective(dataset, verbose_eval=True):
    lgb.train(
        params=lgb_params, 
        train_set=dataset,
        valid_sets=[dataset],
        verbose_eval=verbose_eval,
        fobj=unbiased_lambdarank_objective
    )    


t_plus = np.ones(train_dataset.calculator.max_rank)
t_minus = np.ones(train_dataset.calculator.max_rank)
fit_custom_objective(train_dataset)
print(t_plus, t_minus)

[1]	training's ndcg@10: 0.727797
[2]	training's ndcg@10: 0.72874
[3]	training's ndcg@10: 0.729093
[4]	training's ndcg@10: 0.729409
[5]	training's ndcg@10: 0.728661
[6]	training's ndcg@10: 0.72956
[7]	training's ndcg@10: 0.730146
[8]	training's ndcg@10: 0.729872
[9]	training's ndcg@10: 0.730273
[10]	training's ndcg@10: 0.730882
[1.         0.50126325 0.31899954 0.24712254 0.20910728 0.15146912
 0.13427764 0.11128859 0.08859075 0.08483076] [1.         1.0096045  0.9788292  0.99767647 0.96291405 0.9822002
 0.96958388 1.01251736 1.02118098 1.0175265 ]


#### Using fixed t_plus and t_minus

In [20]:
def get_unbiased_gradient_for_one_query(
    gains,
    preds,
    t_plus,
    t_minus,
    ranks0,
    start, 
    end,
    inverse_max_dcg,
    grad, 
    hess, 
    calculator
):
    sorted_idx = np.argsort(-preds[start:end])
    should_adjust = preds[start + sorted_idx[0]] != preds[start + sorted_idx[-1]]
    for i in range(len(sorted_idx)):
        high = sorted_idx[i]
        gain_high = gains[start + high]
        for j in range(len(sorted_idx)):
            low = sorted_idx[j]
            gain_low = gains[start + low]
            if gain_high > gain_low:
                score_high = preds[start + high]
                score_low = preds[start + low]
                rank_high = ranks0[start + high]
                rank_low = ranks0[start + low]

                score_diff = score_high - score_low
                p_lambda = calculator.get_sigmoid(score_diff)
                p_hess = p_lambda * (2.0 - p_lambda)
                gain_diff = gain_high - gain_low
                paired_discount = calculator.discounts[1 + i] - calculator.discounts[1 + j]
                abs_delta_ndcg = np.abs(gain_diff * paired_discount * inverse_max_dcg)
                if should_adjust == 1:
                    abs_delta_ndcg /= (0.01 + np.abs(score_diff))
                p_lambda *= -abs_delta_ndcg
                p_hess *= 2 * abs_delta_ndcg
                
                p_lambda /= t_plus[rank_high] * t_minus[rank_low]
                p_hess /= t_plus[rank_high] * t_minus[rank_low]
    
                grad[start + high] += p_lambda
                grad[start + low] -= p_lambda
                hess[start + high] += p_hess
                hess[start + low] += p_hess  
                
                
def get_unbiased_gradients(
    gains, 
    preds, 
    t_plus, 
    t_minus,
    ranks0,
    groups, 
    grad_for_one_query_func, 
    calculator
):
    grad = [0] * len(gains)
    hess = [0] * len(gains)
    
    query_boundaries = calculator.query_boundaries   

    for i in range(len(query_boundaries) - 1):
        grad_for_one_query_func(gains, 
                                preds,
                                t_plus,
                                t_minus,
                                ranks0,
                                query_boundaries[i], 
                                query_boundaries[i + 1], 
                                calculator.inverse_max_dcgs[i],
                                grad, 
                                hess,
                                calculator)
        
    return grad, hess


def unbiased_lambdarank_objective(preds, train_data):
    """
    Uses global variables t_plus and t_minus.
    """
    global t_plus, t_minus
    
    groups = train_data.get_group()
    
    if len(groups) == 0:
        raise Error("Group/query data should not be empty.")
    else:
        grad, hess = get_unbiased_gradients(
            train_data.label,
            preds,
            t_plus,
            t_minus,
            train_data.ranks0,
            groups,
            get_unbiased_gradient_for_one_query,
            train_data.calculator
        )
        
        return grad, hess

In [21]:
def fit_custom_objective(dataset, verbose_eval=True):
    lgb.train(
        params=lgb_params, 
        train_set=dataset,
        valid_sets=[dataset],
        verbose_eval=verbose_eval,
        fobj=unbiased_lambdarank_objective
    )    


t_plus = 1 / np.arange(1, train_dataset.calculator.max_rank + 1)
t_minus = np.ones(train_dataset.calculator.max_rank)
fit_custom_objective(train_dataset)

[1]	training's ndcg@10: 0.72752
[2]	training's ndcg@10: 0.727348
[3]	training's ndcg@10: 0.72688
[4]	training's ndcg@10: 0.729711
[5]	training's ndcg@10: 0.729438
[6]	training's ndcg@10: 0.728955
[7]	training's ndcg@10: 0.729402
[8]	training's ndcg@10: 0.730952
[9]	training's ndcg@10: 0.731938
[10]	training's ndcg@10: 0.731942


### Using a cython implementation of the unbiased objective

#### With updating t_plus and t_minus

In [22]:
from unbiasedlambdamart.lambdaobj import get_unbiased_gradients
from unbiasedlambdamart.calculator import MIN_ARG, MAX_ARG
from unbiasedlambdamart.objective import DatasetWithCalculatorRanks0AndP

In [23]:
def unbiased_lambdarank_objective(preds, dataset):
    """
    Uses global variables t_plus and t_minus.
    """
    global t_plus, t_minus
    
    groups = dataset.get_group()
    
    if len(groups) == 0:
        raise Error("Group/query data should not be empty.")
    else:
        grad = np.zeros(len(preds))
        hess = np.zeros(len(preds))
        get_unbiased_gradients(
            np.ascontiguousarray(dataset.label, dtype=np.double), 
            np.ascontiguousarray(preds),
            np.ascontiguousarray(dataset.ranks0),
            len(preds),
            np.ascontiguousarray(groups),
            np.ascontiguousarray(dataset.calculator.query_boundaries),
            len(dataset.calculator.query_boundaries) - 1,
            np.ascontiguousarray(dataset.calculator.discounts),
            np.ascontiguousarray(dataset.calculator.inverse_max_dcgs),
            np.ascontiguousarray(dataset.calculator.sigmoids),
            len(dataset.calculator.sigmoids),
            MIN_ARG,
            MAX_ARG,
            dataset.calculator.idx_factor,
            np.ascontiguousarray(dataset.calculator.logs),
            len(dataset.calculator.logs),
            MIN_ARG,
            MAX_ARG,
            dataset.calculator.idx_factor,
            dataset.p,
            dataset.calculator.max_rank,
            np.ascontiguousarray(grad), 
            np.ascontiguousarray(hess),
            np.ascontiguousarray(t_plus),
            np.ascontiguousarray(t_minus)
        )
        
        return grad, hess

In [24]:
def fit_custom_objective(dataset, verbose_eval=True):
    lgb.train(
        params=lgb_params, 
        train_set=dataset,
        valid_sets=[dataset],
        verbose_eval=verbose_eval,
        fobj=unbiased_lambdarank_objective
    )    

train_dataset = DatasetWithCalculatorRanks0AndP(
    MAX_NDCG_POS,
    train_data.position.values,
    0.0,
    data=train_data.drop(["request_id", "position", "c", "relevance", "viewed", "_c"], axis=1),
    label=train_data.c.values,
    group=[N_POSITIONS] * train_data.request_id.nunique(),
    free_raw_data=False
)

t_plus = np.ones(train_dataset.calculator.max_rank)
t_minus = np.ones(train_dataset.calculator.max_rank)
fit_custom_objective(train_dataset)
print(t_plus, t_minus)

[1]	training's ndcg@10: 0.727797
[2]	training's ndcg@10: 0.72874
[3]	training's ndcg@10: 0.729093
[4]	training's ndcg@10: 0.729409
[5]	training's ndcg@10: 0.728661
[6]	training's ndcg@10: 0.72956
[7]	training's ndcg@10: 0.730146
[8]	training's ndcg@10: 0.729872
[9]	training's ndcg@10: 0.730273
[10]	training's ndcg@10: 0.730882
[1.         0.50126325 0.31899954 0.24712254 0.20910728 0.15146912
 0.13427764 0.11128859 0.08859075 0.08483076] [1.         1.0096045  0.9788292  0.99767647 0.96291405 0.9822002
 0.96958388 1.01251736 1.02118098 1.0175265 ]


#### Using fixed t_plus and t_minus

In [25]:
from unbiasedlambdamart.lambdaobj import get_unbiased_gradients_fixed_t
from unbiasedlambdamart.calculator import MIN_ARG, MAX_ARG

In [26]:
def unbiased_lambdarank_objective_fixed_t(preds, dataset):
    """
    Uses global variables t_plus and t_minus.
    """
    global t_plus, t_minus
    
    groups = dataset.get_group()
    
    if len(groups) == 0:
        raise Error("Group/query data should not be empty.")
    else:
        grad = np.zeros(len(preds))
        hess = np.zeros(len(preds))
        get_unbiased_gradients_fixed_t(
            np.ascontiguousarray(dataset.label, dtype=np.double), 
            np.ascontiguousarray(preds),
            np.ascontiguousarray(t_plus),
            np.ascontiguousarray(t_minus),
            np.ascontiguousarray(dataset.ranks0),
            len(preds),
            np.ascontiguousarray(groups),
            np.ascontiguousarray(dataset.calculator.query_boundaries),
            len(dataset.calculator.query_boundaries) - 1,
            np.ascontiguousarray(dataset.calculator.discounts),
            np.ascontiguousarray(dataset.calculator.inverse_max_dcgs),
            np.ascontiguousarray(dataset.calculator.sigmoids),
            len(dataset.calculator.sigmoids),
            MIN_ARG,
            MAX_ARG,
            dataset.calculator.idx_factor,
            np.ascontiguousarray(grad), 
            np.ascontiguousarray(hess)
        )
        
        return grad, hess

In [27]:
def fit_custom_objective(dataset, verbose_eval=True):
    lgb.train(
        params=lgb_params, 
        train_set=dataset,
        valid_sets=[dataset],
        verbose_eval=verbose_eval,
        fobj=unbiased_lambdarank_objective_fixed_t
    )    


t_plus = 1 / np.arange(1, train_dataset.calculator.max_rank + 1)
t_minus = np.ones(train_dataset.calculator.max_rank)
fit_custom_objective(train_dataset)

[1]	training's ndcg@10: 0.72752
[2]	training's ndcg@10: 0.727348
[3]	training's ndcg@10: 0.72688
[4]	training's ndcg@10: 0.729711
[5]	training's ndcg@10: 0.729438
[6]	training's ndcg@10: 0.728955
[7]	training's ndcg@10: 0.729402
[8]	training's ndcg@10: 0.730952
[9]	training's ndcg@10: 0.731938
[10]	training's ndcg@10: 0.731942


### Using unbiasedlambdamart.objective

#### With updating t_plus and t_minus

In [28]:
from unbiasedlambdamart.objective import unbiased_lambdarank_objective
from unbiasedlambdamart.objective import DatasetWithCalculatorRanks0AndP

In [29]:
train_data = drop_requests_with_no_interactions(train_data, "c")
train_dataset = DatasetWithCalculatorRanks0AndP(
    MAX_NDCG_POS,
    train_data.position.values,
    0.0,
    data=train_data.drop(["request_id", "position", "c", "relevance", "viewed", "_c"], axis=1),
    label=train_data.c.values,
    group=[N_POSITIONS] * train_data.request_id.nunique(),
    free_raw_data=False
)

In [30]:
def fit_custom_objective(dataset, verbose_eval=True):
    lgb.train(
        params=lgb_params, 
        train_set=dataset,
        valid_sets=[dataset],
        verbose_eval=verbose_eval,
        fobj=unbiased_lambdarank_objective
    )    


fit_custom_objective(train_dataset)
print(train_dataset.t_plus, train_dataset.t_minus)

[1]	training's ndcg@10: 0.727797
[2]	training's ndcg@10: 0.72874
[3]	training's ndcg@10: 0.729093
[4]	training's ndcg@10: 0.729409
[5]	training's ndcg@10: 0.728661
[6]	training's ndcg@10: 0.72956
[7]	training's ndcg@10: 0.730146
[8]	training's ndcg@10: 0.729872
[9]	training's ndcg@10: 0.730273
[10]	training's ndcg@10: 0.730882
[1.         0.50126325 0.31899954 0.24712254 0.20910728 0.15146912
 0.13427764 0.11128859 0.08859075 0.08483076] [1.         1.0096045  0.9788292  0.99767647 0.96291405 0.9822002
 0.96958388 1.01251736 1.02118098 1.0175265 ]


#### Using fixed t_plus and t_minus

In [31]:
from unbiasedlambdamart.objective import unbiased_lambdarank_objective_fixed_t
from unbiasedlambdamart.objective import DatasetWithCalculatorRanks0AndT

In [32]:
train_data = drop_requests_with_no_interactions(train_data, "c")
train_dataset = DatasetWithCalculatorRanks0AndT(
    MAX_NDCG_POS,
    train_data.position.values,
    1.0,
    data=train_data.drop(["request_id", "position", "c", "relevance", "viewed", "_c"], axis=1),
    label=train_data.c.values,
    group=[N_POSITIONS] * train_data.request_id.nunique(),
    free_raw_data=False
)

In [33]:
def fit_custom_objective(dataset, verbose_eval=True):
    lgb.train(
        params=lgb_params, 
        train_set=dataset,
        valid_sets=[dataset],
        verbose_eval=verbose_eval,
        fobj=unbiased_lambdarank_objective_fixed_t
    )    


fit_custom_objective(train_dataset)

[1]	training's ndcg@10: 0.72752
[2]	training's ndcg@10: 0.727348
[3]	training's ndcg@10: 0.72688
[4]	training's ndcg@10: 0.729711
[5]	training's ndcg@10: 0.729438
[6]	training's ndcg@10: 0.728955
[7]	training's ndcg@10: 0.729402
[8]	training's ndcg@10: 0.730952
[9]	training's ndcg@10: 0.731938
[10]	training's ndcg@10: 0.731942
